In [1]:
# import tensorflow as tf
import numpy as np
# import keras
# from keras.models import Sequential
# from keras.layers import Dense
import os
import skimage.data
import matplotlib.pyplot as plt
import cv2
%matplotlib inline
import json
import PIL
from PIL import Image

In [8]:
def load_data(data_directory):
    file_names = [os.path.join(data_directory, d) for d in os.listdir(data_directory)]
    images = dict()
    for f in file_names:
#         images.append(skimage.data.imread(f))
        images[f.split('/')[-1][0:-4]] = skimage.data.imread(f)
    return images

In [16]:
resize_ratio = 0.15625
# data_directory = "DataSet/Color/"
# file_names = [os.path.join(data_directory, d) for d in os.listdir(data_directory)]

# for f in file_names:
#     if 'jpg' in f:
#         basewidth = 300
#         img = Image.open(f)
#         wpercent = (basewidth / float(img.size[0]))
#         hsize = int((float(img.size[1]) * float(wpercent)))
#         img = img.resize((basewidth, hsize), PIL.Image.ANTIALIAS)
#         img.save('Dataset/resize/' + f.split('/')[-1])

In [9]:
images = load_data("DataSet/resize/")

In [14]:
with open('DataSet/annotation.json') as f:
    joints = json.load(f)

In [18]:
# Preprocess labels
# pos = dict()
txt_temp = dict()
offset = 10
for key in joints:
    x_coord = [x[0] for x in joints[key]]
    y_coord = [x[1] for x in joints[key]]
#     pos[key] = [int(min(x_coord)), int(min(y_coord)), int(max(x_coord)), int(max(y_coord))]
    max_x = max(x_coord)*resize_ratio
    min_x = min(x_coord)*resize_ratio
    max_y = max(y_coord)*resize_ratio
    min_y = min(y_coord)*resize_ratio
    ax = (min_x + max_x)/2
    ay = (min_y + max_y)/2
    width = images[key[:-2]].shape[1]
    height = images[key[:-2]].shape[0]
    
    txt_temp[key] = [ax/width, ay/height,
                     (max_x - min_x)/width, (max_y - min_y)/height]

In [28]:
# Generate label files for yolo training
for key in images:
    f = open("yolo_labels/"+key+".txt","w+")
    has_L = False
    if key+'_L' in txt_temp.keys():
        value = "{} {} {} {}".format(txt_temp[key+"_L"][0], txt_temp[key+"_L"][1], txt_temp[key+"_L"][2], txt_temp[key+"_L"][3])
        f.write("0 " + value)
        has_L = True
    if key+'_R' in txt_temp.keys():
        value = "{} {} {} {}".format(txt_temp[key+"_R"][0], txt_temp[key+"_R"][1], txt_temp[key+"_R"][2], txt_temp[key+"_R"][3])
        if has_L:
            f.write("\n")
        f.write("1 " + value)
    f.close()